In [1]:
import pandas as pd
import sqlite3

In [2]:
# Load orders CSV
orders = pd.read_csv("data/orders.csv")
print(orders.head())


   order_id  user_id  restaurant_id  order_date  total_amount
0         1     2508            450  18-02-2023        842.97
1         2     2693            309  18-01-2023        546.68
2         3     2084            107  15-07-2023        163.93
3         4      319            224   4/10/2023       1155.97
4         5     1064            293  25-12-2023       1321.91


In [3]:
# Load users JSON
users = pd.read_json("data/users.json")
print(users.head())

   user_id    name       city membership
0        1  User_1    Chennai    Regular
1        2  User_2       Pune       Gold
2        3  User_3  Bangalore       Gold
3        4  User_4  Bangalore    Regular
4        5  User_5       Pune       Gold


In [4]:
# Load restaurants SQL
conn = sqlite3.connect(":memory:")
with open("data/restaurants.sql", "r") as file:
    sql_script = file.read()

conn.executescript(sql_script)
restaurants = pd.read_sql("SELECT * FROM restaurants", conn)
print(restaurants.head())

   restaurant_id restaurant_name  cuisine  rating
0              1    Restaurant_1  Chinese     4.8
1              2    Restaurant_2   Indian     4.1
2              3    Restaurant_3  Mexican     4.3
3              4    Restaurant_4  Chinese     4.1
4              5    Restaurant_5  Chinese     4.8


In [5]:
# -----------------------------
# MERGE DATA (LEFT JOIN)
# -----------------------------

final_df = (
    orders
    .merge(users, on="user_id", how="left")
    .merge(restaurants, on="restaurant_id", how="left")
)

In [6]:
# FIX mixed date formats
final_df["order_date"] = pd.to_datetime(
    final_df["order_date"],
    dayfirst=True,
    format="mixed",
    errors="coerce"
)
print("\nFINAL DATASET PREVIEW:")
print(final_df.head())
# Save final dataset
final_df.to_csv("final_food_delivery_dataset.csv", index=False)


FINAL DATASET PREVIEW:
   order_id  user_id  restaurant_id order_date  total_amount       name  \
0         1     2508            450 2023-02-18        842.97  User_2508   
1         2     2693            309 2023-01-18        546.68  User_2693   
2         3     2084            107 2023-07-15        163.93  User_2084   
3         4      319            224 2023-10-04       1155.97   User_319   
4         5     1064            293 2023-12-25       1321.91  User_1064   

        city membership restaurant_name  cuisine  rating  
0  Hyderabad    Regular  Restaurant_450  Mexican     3.2  
1       Pune    Regular  Restaurant_309   Indian     4.5  
2    Chennai       Gold  Restaurant_107  Mexican     4.0  
3  Bangalore       Gold  Restaurant_224  Chinese     4.8  
4       Pune    Regular  Restaurant_293  Italian     3.0  


In [7]:
# Q1: City with highest total revenue from Gold members
q1 = (
    final_df[final_df["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .sum()
)
print("\nQ1 - Gold Revenue by City:")
print(q1)


Q1 - Gold Revenue by City:
city
Bangalore     994702.59
Chennai      1080909.79
Hyderabad     896740.19
Pune         1003012.32
Name: total_amount, dtype: float64


In [8]:
# Q2: Cuisine with highest average order value
q2 = (
    final_df
    .groupby("cuisine")["total_amount"]
    .mean()
)

print("\nQ2 - Average Order Value by Cuisine:")
print(q2)


Q2 - Average Order Value by Cuisine:
cuisine
Chinese    798.389020
Indian     798.466011
Italian    799.448578
Mexican    808.021344
Name: total_amount, dtype: float64


In [9]:
# Q3: Distinct users with total order value > 1000
user_total = (
    final_df
    .groupby("user_id")["total_amount"]
    .sum()
)

count_users = (user_total > 1000).sum()

print("\nQ3 - Number of users with total spend > 1000:")
print(count_users)


Q3 - Number of users with total spend > 1000:
2544


In [10]:
# Q4: Rating range with highest total revenue

def rating_bucket(r):
    if 3.0 <= r <= 3.5:
        return "3.0 – 3.5"
    elif 3.6 <= r <= 4.0:
        return "3.6 – 4.0"
    elif 4.1 <= r <= 4.5:
        return "4.1 – 4.5"
    else:
        return "4.6 – 5.0"

final_df["rating_range"] = final_df["rating"].apply(rating_bucket)

q4 = (
    final_df
    .groupby("rating_range")["total_amount"]
    .sum()
)

print("\nQ4 - Total Revenue by Rating Range:")
print(q4)


Q4 - Total Revenue by Rating Range:
rating_range
3.0 – 3.5    2136772.70
3.6 – 4.0    1717494.41
4.1 – 4.5    1960326.26
4.6 – 5.0    2197030.75
Name: total_amount, dtype: float64


In [11]:
# Q5: Among Gold members, city with highest average order value

q5 = (
    final_df[final_df["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .mean()
)

print("\nQ5 - Average Order Value (Gold Members) by City:")
print(q5)


Q5 - Average Order Value (Gold Members) by City:
city
Bangalore    793.223756
Chennai      808.459080
Hyderabad    806.421034
Pune         781.162243
Name: total_amount, dtype: float64


In [12]:
# Q6: Cuisine with lowest number of distinct restaurants but significant revenue

cuisine_restaurant_count = (
    final_df
    .groupby("cuisine")["restaurant_id"]
    .nunique()
)

cuisine_revenue = (
    final_df
    .groupby("cuisine")["total_amount"]
    .sum()
)

q6 = (
    cuisine_restaurant_count
    .to_frame("restaurant_count")
    .join(cuisine_revenue.to_frame("total_revenue"))
)

print("\nQ6 - Restaurant Count and Revenue by Cuisine:")
print(q6)


Q6 - Restaurant Count and Revenue by Cuisine:
         restaurant_count  total_revenue
cuisine                                 
Chinese               120     1930504.65
Indian                126     1971412.58
Italian               126     2024203.80
Mexican               128     2085503.09


In [13]:
# Q7: Percentage of orders by Gold members
total_orders = len(final_df)
gold_orders = len(final_df[final_df["membership"] == "Gold"])

percentage_gold = (gold_orders / total_orders) * 100

print("\nQ7 - Percentage of orders by Gold members:")
print(round(percentage_gold))


Q7 - Percentage of orders by Gold members:
50


In [14]:
# Q8: Restaurant with highest AOV but < 20 orders
restaurant_stats = (
    final_df
    .groupby("restaurant_name")
    .agg(
        avg_order_value=("total_amount", "mean"),
        order_count=("order_id", "count")
    )
)

q8 = restaurant_stats[restaurant_stats["order_count"] < 20] \
        .sort_values("avg_order_value", ascending=False)

print("\nQ8 - Restaurants with <20 orders sorted by AOV:")
print(q8.head())


Q8 - Restaurants with <20 orders sorted by AOV:
                 avg_order_value  order_count
restaurant_name                              
Restaurant_294       1040.222308           13
Restaurant_262       1029.473333           18
Restaurant_77        1029.180833           12
Restaurant_193       1026.306667           15
Restaurant_7         1002.140625           16


In [15]:
# Q9: Revenue by membership + cuisine
q9 = (
    final_df
    .groupby(["membership", "cuisine"])["total_amount"]
    .sum()
)

print("\nQ9 - Revenue by Membership & Cuisine:")
print(q9)


Q9 - Revenue by Membership & Cuisine:
membership  cuisine
Gold        Chinese     977713.74
            Indian      979312.31
            Italian    1005779.05
            Mexican    1012559.79
Regular     Chinese     952790.91
            Indian      992100.27
            Italian    1018424.75
            Mexican    1072943.30
Name: total_amount, dtype: float64


In [16]:
# Q10: During which quarter of the year is the total revenue highest?

# Extract quarter from order_date
final_df["quarter"] = final_df["order_date"].dt.to_period("Q")

q10 = (
    final_df
    .groupby("quarter")["total_amount"]
    .sum()
)

print("\nQ10 - Total Revenue by Quarter:")
print(q10)


Q10 - Total Revenue by Quarter:
quarter
2023Q1    1993425.14
2023Q2    1945348.72
2023Q3    2037385.10
2023Q4    2018263.66
2024Q1      17201.50
Freq: Q-DEC, Name: total_amount, dtype: float64


In [17]:
# NUMERICAL QUESTIONS (FORM ANSWERS)

# 1. Total orders placed by Gold members
total_gold_orders = final_df[final_df["membership"] == "Gold"].shape[0]
print("\nANS 1 - Total orders by Gold members:")
print(total_gold_orders)


ANS 1 - Total orders by Gold members:
4987


In [18]:
# 2. Total revenue from Hyderabad city (rounded)
hyderabad_revenue = (
    final_df[final_df["city"] == "Hyderabad"]["total_amount"]
    .sum()
)

print("\nANS 2 - Total revenue from Hyderabad (rounded):")
print(round(hyderabad_revenue))



ANS 2 - Total revenue from Hyderabad (rounded):
1889367


In [19]:
# 3. Total distinct users who placed at least one order
distinct_users = final_df["user_id"].nunique()

print("\nANS 3 - Distinct users with at least one order:")
print(distinct_users)



ANS 3 - Distinct users with at least one order:
2883


In [20]:
# 4. Average order value for Gold members (rounded to 2 decimals)
gold_avg_order_value = (
    final_df[final_df["membership"] == "Gold"]["total_amount"]
    .mean()
)

print("\nANS 4 - Average order value for Gold members:")
print(round(gold_avg_order_value, 2))



ANS 4 - Average order value for Gold members:
797.15


In [21]:

# 5. Orders placed for restaurants with rating >= 4.5
high_rating_orders = final_df[final_df["rating"] >= 4.5].shape[0]

print("\nANS 5 - Orders with restaurant rating >= 4.5:")
print(high_rating_orders)


ANS 5 - Orders with restaurant rating >= 4.5:
3374


In [22]:
# 6. Orders in top revenue city among Gold members
top_gold_city = (
    final_df[final_df["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .sum()
    .idxmax()
)

orders_top_gold_city = (
    final_df[
        (final_df["membership"] == "Gold") &
        (final_df["city"] == top_gold_city)
    ]
    .shape[0]
)

print("\nANS 6 - Orders in top revenue city among Gold members:")
print(orders_top_gold_city)
print("Top Gold Revenue City:", top_gold_city)



ANS 6 - Orders in top revenue city among Gold members:
1337
Top Gold Revenue City: Chennai
